## 按行业处理A股数据
整理整个处理的逻辑
- 处理数据
    - 去除ST
    - 更改列名
    - 去除全为空的列
    - 去除不连续的公司
    - 导出数据，roeData.xlsx
    - winsorize
    - 导出数据，winsorized.xlsx
- 计算行业标准差，取得波动大于行业标准差的公司，roeStd，导出数据，
- 从roeStd中再根据规则筛选出good，规则如超过n个std的比例超过proportion
- 对good进行mark，导出mark数据